# Guidelines for Prompting
In this lesson, you'll practice two prompting principles and their related tactics in order to write effective prompts for large language models.

## Setup
#### Load the API key and relevant Python libaries.

## Setup
Load the API key

In [1]:
using DotEnv
using HTTP, JSON

In [2]:
DotEnv.load()
api_key = ENV["OPENAI_API_KEY"];

## Helper Function

Documentation to make the HTTP request:
https://platform.openai.com/docs/api-reference/making-requests

In [10]:
using HTTP, JSON

function get_completion(prompt, api_key=api_key, temperature=0)
    url = "https://api.openai.com/v1/chat/completions"
    headers = [
        "Content-Type" => "application/json",
        "Authorization" => "Bearer $api_key"
    ]
    data = Dict(
        "model" => "gpt-3.5-turbo",
        "messages" => [Dict("role" => "user", "content" => prompt)],
        "temperature" => temperature
    )
    body = JSON.json(data)
    response = HTTP.post(url, headers=headers, body=body)
    response_dict = JSON.parse(String(response.body))

    # Extract the message
    message = response_dict["choices"][1]["message"]["content"]

    return message
end

get_completion (generic function with 3 methods)

## Prompting Principles
- **Principle 1: Write clear and specific instructions**
- **Principle 2: Give the model time to “think”**

### Tactics

#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`

## Notes specific to Julia:
- Instead of using {  } for string interpolation with f strings in Python, __$__ is used in Julia.
- Don't use single backslash, \\, in the text or you will get an "Invalid escape character" error. 
Either use double backslash, \\\\, or don't use it at all. 
- If you have to use \$ sign in a text, write it as \\$ as it is the string interpolation operator in Julia. 
- In Julia, " is used for strings and ' is used for characters. 

In [11]:
text = """
You should express what you want a model to do by  
providing instructions that are as clear and  
specific as you can possibly make them.  
This will guide the model towards the desired output,  
and reduce the chances of receiving irrelevant  
or incorrect responses. Don't confuse writing a  
clear prompt with writing a short prompt.  
In many cases, longer prompts provide more clarity  
and context for the model, which can lead to  
more detailed and relevant outputs.
"""
prompt = """
Summarize the text delimited by triple backticks 
into a single sentence.
```$(text)```
"""

response = get_completion(prompt, api_key)
println(response)

To guide a model towards the desired output and reduce the chances of irrelevant or incorrect responses, it is important to provide clear and specific instructions, which may require longer prompts for more clarity and context.


#### Tactic 2: Ask for a structured output
- JSON, HTML

In [12]:
prompt = """
Please generate a list of three completely made-up and creative book titles, 
along with their fictional authors and imaginative genres in JSON format. 
Make sure that the titles, authors, and genres have never been used in 
real life and are purely imaginative.

Example:
[{
  "book_id": 1,
  "title": "Example Imaginary Title",
  "author": "Example Fictional Author",
  "genre": "Example Creative Genre"
}]
"""
response = get_completion(prompt, api_key, 0.9)
println(response)

[
  {
    "book_id": 1,
    "title": "The Last Flutist",
    "author": "Evelyn Moonstone",
    "genre": "Fantasy/Adventure"
  },
  {
    "book_id": 2,
    "title": "The Quantum Clockmaker's Daughter",
    "author": "Caleb Fig",
    "genre": "Science Fiction/Romance"
  },
  {
    "book_id": 3,
    "title": "The Enchanted Waffle Iron",
    "author": "Zara Bluebird",
    "genre": "Children's Fiction/Fantasy"
  }
]


#### Tactic 3: Ask the model to check whether conditions are satisfied

In [16]:
text_1 = """
Making a cup of tea is easy! First, you need to get some 
water boiling. While that's happening, 
grab a cup and put a tea bag in it. Once the water is 
hot enough, just pour it over the tea bag. 
Let it sit for a bit so the tea can steep. After a 
few minutes, take out the tea bag. If you 
like, you can add some sugar or milk to taste. 
And that's it! You've got yourself a delicious 
cup of tea to enjoy.
"""

prompt = """
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions,  
then simply write "No steps provided."

\\"\\"\\"$(text_1)\\"\\"\\"
"""
response = get_completion(prompt, api_key, 0.7)
print("Completion for Text 1:")
print(response)

Completion for Text 1:Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Once the water is hot enough, pour it over the tea bag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - After a few minutes, take out the tea bag.
Step 6 - Add some sugar or milk to taste.
Step 7 - Enjoy your delicious cup of tea!

In [17]:
text_2 = """
The sun is shining brightly today, and the birds are
singing. It's a beautiful day to go for a 
walk in the park. The flowers are blooming, and the 
trees are swaying gently in the breeze. People 
are out and about, enjoying the lovely weather. 
Some are having picnics, while others are playing 
games or simply relaxing on the grass. It's a 
perfect day to spend time outdoors and appreciate the 
beauty of nature.
"""
prompt = """
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions,  
then simply write "No steps provided."

\\"\\"\\"$text_2\\"\\"\\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:

No steps provided.

#### Tactic 4: "Few-shot" prompting

In [18]:
prompt = """
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest 
valley flows from a modest spring; the 
grandest symphony originates from a single note; 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: Resilience is like a tree that bends but does not break in the face of strong winds. It is the ability to bounce back from adversity and to keep moving forward despite challenges. Just like a tree, we must have strong roots and a solid foundation to weather any storm that comes our way.

### Principle 2: Give the model time to “think” 

#### Tactic 1: Specify the steps required to complete a task

In [20]:
text = """
In a charming village, siblings Jack and Jill set out on 
a quest to fetch water from a hilltop 
well. As they climbed, singing joyfully, misfortune6                                               " 
struck—Jack tripped on a stone and tumbled 
down the hill, with Jill following suit. 
Though slightly battered, the pair returned home to 
comforting embraces. Despite the mishap, 
their adventurous spirits remained undimmed, and they 
continued exploring with delight.
"""
# example 1
prompt_1 = """
Perform the following actions: 
1 - Summarize the following text delimited by triple
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{$text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:In a charming village, siblings Jack and Jill go on a quest to fetch water from a hilltop well, but misfortune strikes as they both tumble down the hill, yet they return home slightly battered but with their adventurous spirits undimmed.

Dans un charmant village, Jack et Jill partent en quête d'eau d'un puits au sommet d'une colline, mais malheureusement, ils tombent tous les deux de la colline, mais ils rentrent chez eux légèrement meurtris mais avec leurs esprits d'aventure intacts. 
Noms: Jack, Jill.

{
"french_summary": "Dans un charmant village, Jack et Jill partent en quête d'eau d'un puits au sommet d'une colline, mais malheureusement, ils tombent tous les deux de la colline, mais ils rentrent chez eux légèrement meurtris mais avec leurs esprits d'aventure intacts.",
"num_names": 2
}

#### Ask for output in a specified format

In [21]:
prompt_2 = """
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in Italian summary>
Output JSON: <json with summary and num_names>

Text: <$text>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:Summary: Jack and Jill go on a quest to fetch water from a hilltop well, but misfortune strikes and they both tumble down the hill, returning home slightly battered but with their adventurous spirits undimmed. 
Translation: Jack et Jill partent en quête d'eau d'un puits au sommet d'une colline, mais la malchance frappe et ils tombent tous les deux, rentrant chez eux légèrement meurtris mais avec leurs esprits aventureux intacts.
Names: Jack, Jill
Output JSON: {"french_summary": "Jack et Jill partent en quête d'eau d'un puits au sommet d'une colline, mais la malchance frappe et ils tombent tous les deux, rentrant chez eux légèrement meurtris mais avec leurs esprits aventureux intacts.", "num_names": 2}

#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [23]:
prompt = """
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need
 help working out the financials. 
- Land costs \$100 / square foot
- I can buy solar panels for \$250 / square foot
- I negotiated a contract for maintenance that will cost 
me a flat \$100k per year, and an additional \$10 / square
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is correct.

Note that the student's solution is actually not correct.

We can fix this by instructing the model to work out its own solution first.

In [24]:
prompt = """
Your task is to determine if the student's solution
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help
working out the financials. 
- Land costs \$100 / square foot
- I can buy solar panels for \$250 / square foot
- I negotiated a contract for maintenance that will cost
me a flat \$100k per year, and an additional \$10 / square
foot
What is the total cost for the first year of operations
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 10x
Total cost: 100x + 250x + 100,000 + 10x = 360x + 100,000

Is the student's solution the same as actual solution
just calculated:
no

Student grade:
incorrect

## Model Limitations: Hallucinations
- Boie is a real company, the product name is not real.

In [26]:
prompt = """
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""
response = get_completion(prompt)
print(response)

The AeroGlide UltraSlim Smart Toothbrush by Boie is a high-tech toothbrush that uses advanced sonic technology to provide a deep and thorough clean. The toothbrush features a slim and sleek design that makes it easy to hold and maneuver, and it comes with a range of smart features that help you optimize your brushing routine.

One of the key features of the AeroGlide UltraSlim Smart Toothbrush is its sonic technology, which uses high-frequency vibrations to break up plaque and bacteria on your teeth and gums. This technology is highly effective at removing even the most stubborn stains and buildup, and it can help you achieve a brighter, healthier smile in just a few weeks.

In addition to its sonic technology, the AeroGlide UltraSlim Smart Toothbrush also comes with a range of smart features that help you optimize your brushing routine. These features include a built-in timer that ensures you brush for the recommended two minutes, as well as a pressure sensor that alerts you if you're